In [10]:
import pandas as pd
import numpy as np

In [11]:
#read in data
account_dim = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/account_dimensions.csv')
case_notes = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/support_cases_text_fact.csv')
support_fact = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/support_cases_fact.csv')
oppertunity_fact = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/opportunity_fact.csv')
date_dim = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/date_dimensions.csv')
opportunity_summary = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/TFL_PivotedData.csv')
tenure = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/account_tenure_by_month.csv',  encoding = 'unicode_escape', engine ='python')

In [12]:
#join supoprt tickets with account_dim
support_account = support_fact.merge(account_dim,
                                     on='ACCOUNTKEY',
                                     how='left')
support_account = support_account.drop_duplicates(subset='CASE_NUMBER')

print(support_account['CASE_NUMBER'].value_counts().max())
print(support_account.shape)

1
(205061, 27)


In [13]:
#join support_account table with tenure_dim
support_accounts = support_account.merge(tenure,
                                         on='ACCOUNTKEY',
                                         how='left')
#dropping duplicated from left join
support_accounts = support_accounts.drop_duplicates(subset='CASE_NUMBER')
#dropping useless cols
support_accounts = support_accounts.drop(['Unnamed: 0_x', 'PREFERRED_CASE_LANGUAGE',
                                          'Unnamed: 0_y', 'REGION_y', 'REGION_x',
                                          'PARENTACCOUNTKEY', 'TOPPARENTACCOUNTKEY',
                                          'ï..Tenure.Date.Value', 'Unnamed: 0'], axis =1)
#filter support_accounts
support_accounts = support_accounts[(support_accounts['ACCOUNTKEY'].isna() != True) &
                                    (support_accounts['Zero.Revenue.Flag'].isna() != True)]
#define a dict to fill NAs
fill_dict = {'PRIORITY':'None',
             'IMPACT': 'None',
             'ORIGIN': 'Other',
             'PRODUCT': 'Other',
             'CATEGORY': 'Other',
             'SUBCATEGORY': 'None',
             'CASE_CLOSED_REASON': 'None',
             'EMPLOYEECOUNT': 0}
support_accounts.fillna(value = fill_dict, inplace=True)
#sanity check
print(support_accounts.shape)
print(support_accounts.columns)
print(support_accounts.isna().sum())

(183669, 22)
Index(['STATUS', 'PRIORITY', 'IMPACT', 'ORIGIN', 'PRODUCT', 'CATEGORY',
       'SUBCATEGORY', 'CREATED_DATE', 'CLOSED_DATE', 'CASE_CLOSED_REASON',
       'RECORDTYPE', 'CASE_NUMBER', 'ACCOUNTKEY', 'ACCOUNTTYPE',
       'ACCOUNTTYPEDETAIL', 'EMPLOYEECOUNT', 'THEATER', 'SUBREGION', 'SEGMENT',
       'SUBSEGMENT', 'Tenure', 'Zero.Revenue.Flag'],
      dtype='object')
STATUS                0
PRIORITY              0
IMPACT                0
ORIGIN                0
PRODUCT               0
CATEGORY              0
SUBCATEGORY           0
CREATED_DATE          0
CLOSED_DATE           0
CASE_CLOSED_REASON    0
RECORDTYPE            0
CASE_NUMBER           0
ACCOUNTKEY            0
ACCOUNTTYPE           0
ACCOUNTTYPEDETAIL     0
EMPLOYEECOUNT         0
THEATER               0
SUBREGION             0
SEGMENT               0
SUBSEGMENT            0
Tenure                0
Zero.Revenue.Flag     0
dtype: int64


In [14]:
#filter opertunities 
closed_sales = oppertunity_fact[(oppertunity_fact['PRODUCTTYPE'] != 'Maintenance Renewal') &
                                (oppertunity_fact['PRODUCTTYPE'] != 'Licence Renewal') &
                                (oppertunity_fact['ISCLOSED'] == True) &
                                (oppertunity_fact['COMPARATIVESALESCREDITAMOUNT'] >0) &
                                (oppertunity_fact['TERMLENGTH'] >0) &
                                (oppertunity_fact['PRODUCTGROUP'].isna() != True) &
                                (oppertunity_fact['ACCOUNTKEY'].isna() != True)]
#dropping useless cols
closed_sales = closed_sales.drop(['ORIGINAL_SALES_DATE', 'ISDELETED', 'ISPARTNERRELATED',
                                  'CAPACITYCOUNT', 'MAINTENANCESTARTDATE', 
                                  'MAINTENANCEENDDATE', 'PRODUCTTYPEOVERALL',
                                  'PRODUCTCATEGORY', 'STAGENAME', 'CONSOLIDATEDREASON',
                                  'PARTNERACCOUNTKEY', 'CONVERSIONOPPORTUNITYID'], axis =1)
print(closed_sales.columns)
print(closed_sales.shape)
print(closed_sales.isna().sum())

Index(['Unnamed: 0', 'CREATEDDATE', 'CLOSEDATE', 'ISUPGRADE', 'ISCLOSED',
       'ISWON', 'NUMBEROFDESKTOPLICENSES', 'NUMBEROFINTERACTORS',
       'NUMBEROFCORES', 'QUANTITY', 'TERMLENGTH',
       'COMPARATIVESALESCREDITAMOUNT', 'ACV', 'PRODUCTTYPE', 'PRODUCTGROUP',
       'PRODUCTSUBCATEGORY', 'IS_INCLUDED_IN_ATTAINMENT', 'ACCOUNTKEY',
       'OPPORTUNITYID', 'OPPORTUNITYLINEITEMID'],
      dtype='object')
(1268070, 20)
Unnamed: 0                      0
CREATEDDATE                     0
CLOSEDATE                       0
ISUPGRADE                       0
ISCLOSED                        0
ISWON                           0
NUMBEROFDESKTOPLICENSES         0
NUMBEROFINTERACTORS             0
NUMBEROFCORES                   0
QUANTITY                        0
TERMLENGTH                      0
COMPARATIVESALESCREDITAMOUNT    0
ACV                             0
PRODUCTTYPE                     0
PRODUCTGROUP                    0
PRODUCTSUBCATEGORY              0
IS_INCLUDED_IN_ATTAINMENT      

In [15]:
#closed_sales.to_csv('/Users/michaelcolellajensen/Desktop/Tableau/cleaned_oppertunity_fact.csv')
#support_accounts.to_csv('/Users/michaelcolellajensen/Desktop/Tableau/cleaned_suportcases_accounts.csv')

In [16]:
closed_sales['CLOSEDATE'] = pd.to_datetime(closed_sales['CLOSEDATE'])
recent_sales = (
    closed_sales.sort_values('CLOSEDATE', ascending=False).reset_index(drop=True).groupby('ACCOUNTKEY')
)
new_sales = closed_sales.loc[closed_sales.groupby('ACCOUNTKEY')['CLOSEDATE'].idxmax()]
print(new_sales.shape)

(134091, 20)


In [17]:
#new_sales.to_csv('/Users/michaelcolellajensen/Desktop/Tableau/most_receent_oppertunity.csv')

In [24]:
print(opportunity_summary.shape)
print(opportunity_summary['ACCOUNTKEY'].nunique())
print(opportunity_summary.columns)

(290216, 13)
105728
Index(['Einstein Analytics', 'Role Based - Online', 'Server', 'Saas',
       'Role Based - On Prem', 'Services', 'Desktop', 'Role Based - NonProd',
       'Other', 'ACCOUNTKEY', 'CLOSEDATE', 'ISWON', 'ISPARTNERRELATED'],
      dtype='object')


In [32]:
support_accounts['CLOSED_DATE'] = pd.to_datetime(support_accounts['CLOSED_DATE'])
opportunity_summary['CLOSEDATE'] = pd.to_datetime(opportunity_summary['CLOSEDATE'])
support_accounts = support_accounts.sort_values('CLOSED_DATE')
opportunity_summary = opportunity_summary.sort_values('CLOSEDATE')
support_oppertunity = pd.merge_asof(left = support_accounts, 
                                    right = opportunity_summary, 
                                    left_on ='CLOSED_DATE',
                                    right_on = 'CLOSEDATE',
                                    by = 'ACCOUNTKEY',
                                    direction='forward',
                                    tolerance= pd.Timedelta('60d')
                                    )
print(support_oppertunity.shape)

(183669, 34)


In [38]:
support_oppertunity.fillna(0, inplace=True)
print(support_oppertunity['ACCOUNTKEY'].nunique())
print(support_oppertunity['CASE_NUMBER'].nunique())
print(support_oppertunity.isna().sum())
print(support_oppertunity.head())

28348
183669
STATUS                  0
PRIORITY                0
IMPACT                  0
ORIGIN                  0
PRODUCT                 0
CATEGORY                0
SUBCATEGORY             0
CREATED_DATE            0
CLOSED_DATE             0
CASE_CLOSED_REASON      0
RECORDTYPE              0
CASE_NUMBER             0
ACCOUNTKEY              0
ACCOUNTTYPE             0
ACCOUNTTYPEDETAIL       0
EMPLOYEECOUNT           0
THEATER                 0
SUBREGION               0
SEGMENT                 0
SUBSEGMENT              0
Tenure                  0
Zero.Revenue.Flag       0
Einstein Analytics      0
Role Based - Online     0
Server                  0
Saas                    0
Role Based - On Prem    0
Services                0
Desktop                 0
Role Based - NonProd    0
Other                   0
CLOSEDATE               0
ISWON                   0
ISPARTNERRELATED        0
dtype: int64
             STATUS PRIORITY             IMPACT  ORIGIN          PRODUCT  \
0  Pending Cus

In [39]:
#support_oppertunity.to_csv('/Users/michaelcolellajensen/Desktop/Tableau/case_oppertunities.csv')